In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 30
verbosity = 2
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.ResNet152(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

234700800/234698864 [==============================] - 36s 0us/step


In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.
Found 386 images belonging to 2 classes.


In [5]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

Epoch 1/30
40/40 - 32s - loss: 3.5831 - auc: 0.4926 - accuracy: 0.4833 - val_loss: 0.4347 - val_auc: 0.7156 - val_accuracy: 0.8307
Epoch 2/30
40/40 - 27s - loss: 1.0242 - auc: 0.5705 - accuracy: 0.5419 - val_loss: 0.9327 - val_auc: 0.7593 - val_accuracy: 0.3594
Epoch 3/30
40/40 - 27s - loss: 0.8017 - auc: 0.6437 - accuracy: 0.6061 - val_loss: 0.7060 - val_auc: 0.8054 - val_accuracy: 0.5208
Epoch 4/30
40/40 - 28s - loss: 0.7054 - auc: 0.6926 - accuracy: 0.6408 - val_loss: 0.3695 - val_auc: 0.8482 - val_accuracy: 0.8568
Epoch 5/30
40/40 - 28s - loss: 0.7597 - auc: 0.6903 - accuracy: 0.6278 - val_loss: 2.9804 - val_auc: 0.8499 - val_accuracy: 0.1875
Epoch 6/30
40/40 - 26s - loss: 1.3085 - auc: 0.6209 - accuracy: 0.5766 - val_loss: 1.5946 - val_auc: 0.8403 - val_accuracy: 0.3151
Epoch 7/30
40/40 - 27s - loss: 0.6330 - auc: 0.7645 - accuracy: 0.6959 - val_loss: 0.4825 - val_auc: 0.8776 - val_accuracy: 0.7526
Epoch 8/30
40/40 - 27s - loss: 0.7699 - auc: 0.7227 - accuracy: 0.6554 - val_loss: 

In [6]:
prediction = model.predict(valid_generator)

In [7]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

283
103


In [8]:
prediction

array([[6.36836529e-01],
       [6.24748468e-01],
       [9.64605868e-01],
       [9.99976397e-01],
       [8.44504297e-01],
       [1.24316573e-01],
       [6.14718258e-01],
       [9.15161371e-01],
       [9.99503255e-01],
       [8.46565902e-01],
       [2.58816004e-01],
       [4.90389049e-01],
       [2.32973605e-01],
       [3.68838131e-01],
       [4.53181677e-02],
       [6.98607624e-01],
       [9.87302423e-01],
       [8.52750897e-01],
       [5.61599672e-01],
       [8.59445214e-01],
       [9.57774520e-01],
       [9.82736230e-01],
       [2.42825419e-01],
       [8.64767969e-01],
       [8.39905143e-01],
       [9.34032559e-01],
       [6.03693306e-01],
       [3.36819768e-01],
       [3.14860314e-01],
       [9.99729335e-01],
       [9.90870059e-01],
       [9.33843493e-01],
       [9.95914996e-01],
       [9.70146537e-01],
       [9.97601569e-01],
       [9.58369255e-01],
       [9.55671489e-01],
       [9.80610251e-01],
       [9.42806005e-01],
       [9.44521308e-01],


In [9]:
tf.__version__

'2.3.1'

In [10]:
model.evaluate(valid_generator)

7/7 [==============================] - 3s 413ms/step - loss: 0.2752 - auc: 0.9651 - accuracy: 0.8731


[0.27518248558044434, 0.9651041030883789, 0.8730570077896118]